## Topography graphs for Class/frequency CSP


In [ ]:
#!pip install mne
import mne
import numpy as np
from mne.viz import plot_topomap



In [ ]:
# Parâmetros de todos os mapas topográficos:
''' TODO: Guardar índices da feature selection para saber quais as bandas mais discriminativas do participante com melhor acuracia''' 

band = 2 # Alpha
trial = 5


#higlight dos canais principais
h_ch = ['C3..', 'Cz..', 'C4..']

highligted_ch = np.zeros((len(ch)))

for i, c in enumerate(ch):
  for j in h_ch:
    if c == j:
      highligted_ch[i] = 1

highligted_ch = [int(idx) for idx in highligted_ch]

mk = dict(marker='o', markerfacecolor='w', markeredgecolor='r',
        linewidth=0, markersize=5)

## LEFT HAND

In [ ]:
ch = ["Fc1.","Fc5.","C1..","C3..","C5..","Cp1.","Cp5.","Fcz.","Cz..","Fc2.","Fc6.","C2..","C4..","C6..","Cp2.","Cp6."]
fs = 160

info = mne.create_info(ch, fs, ch_types='eeg')

n_trials, _, _ = np.shape(perBand_Z_test[bestBand][1]) #esta perBand_Z_test é um array que inclui a matrix W para cada banda de frequencias para cada classe

alltrials_left_fist = []

for trial in range(n_trials):

  left_fist = mne.io.RawArray(perBand_Z_test[bestBand][1][trial], info)
#   left_fist = change_montage(left_fist,ch)

  alltrials_left_fist.append(left_fist)

_, n_samples = np.shape(left_fist.get_data())

samples = (np.linspace(0, n_samples, n_samples, endpoint=True))
samples = [int(i) for i in samples]

win = int(50/(1000/fs)) # 50 ms em amostras
iter = int(100/(1000/fs))

n_rows = 4
n_cols = 6
fig, axes = plt.subplots(nrows= n_rows, ncols=n_cols, figsize=(24, 16))

count = 0

for s in range(min(samples)+iter, max(samples)-iter, iter):
    
    ax = axes.flat[count]
    count+=1

    ax.set_title(f"Time: {((s)/fs)*1000+500} ms", fontsize=12)
    tstart = ((s-win)/fs)
    tstop = ((s+win)/fs)
    
    aver_psd = []

    for trial in range(n_trials):
      left_fist = alltrials_left_fist[trial]
      aver_psd.append(np.mean(left_fist.compute_psd(method='welch', fmin=bands[bestBand][0], fmax=bands[bestBand][1], 
                                                    tmin=tstart, tmax=tstop, n_per_seg=win*2).get_data(),axis = 1))
    
    data2plot = np.mean(aver_psd, axis = 0)

    plot_topomap(data2plot, left_fist.info, axes=ax, size = 6, sensors=True, 
                 mask = np.array(highligted_ch), mask_params = mk, 
                 names = None, contours=10 ,show = False, cmap = 'rainbow', 
                 sphere=(-0.001, 0.015, 0.025, 0.105));

fig.suptitle("Left Fist PSD", fontweight='bold', fontsize=16)
plt.show()